## установка библиотек, подгрузка API ключа, предобработка входных данных по правилам создателей TimeGPT

In [ ]:
!pip install nixtlats -q # установка модели TimeGPT-1
# метрики
!pip install sktime -q
from sktime.forecasting.model_selection import temporal_train_test_split # для разделения временных рядов на тренировочные и тестовые наборы.
from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError
smape = MeanAbsolutePercentageError(symmetric = True)#метрика sMAP
from sklearn.metrics import mean_squared_error,mean_absolute_error #метрики
from sklearn.model_selection import TimeSeriesSplit #для раздления набора данных
import pandas as pd
from prettytable import PrettyTable # для красоты и порядка

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.0 MB/s eta 0:00:00


In [ ]:

!pip install --upgrade gdown #https://github.com/wkentaro/gdown/issues/333
!git clone https://github.com/zhouhaoyi/ETDataset # офицальный репозиторий температуры машинного масла
!gdown --fuzzy https://drive.google.com/file/d/1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM/view?usp=drive_link
!unzip /content/weather.zip
!unzip /content/exchange_rate.zip
!unzip /content/illness.zip

Cloning into 'ETDataset'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 187 (delta 25), reused 20 (delta 20), pack-reused 159
Receiving objects: 100% (187/187), 3.86 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Downloading...
From: https://drive.google.com/uc?id=1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr
To: /content/weather.zip
100% 2.29M/2.29M [00:00<00:00, 120MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK
To: /content/exchange_rate.zip
100% 194k/194k [00:00<00:00, 84.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM
To: /content/illness.zip
100% 26.0k/26.0k [00:00<00:00, 43.8MB/s]
Archive:  /content/weather.zip
   creating: weather/
  inflating: weather/weather.csv     
Archive:  /content/exchange_rate.zip
   creating: exchange_rate/
  inflating: exchange_rate/.DS_Store  
  inflating: ex

In [ ]:
from google.colab import files
uploaded = files.upload() # подгрузка токена в виртуальную среду

Saving diplom.json to diplom.json


In [ ]:
import json
with open('diplom.json') as json_file:
    data = json.load(json_file)
    api_key = data['key'] # передаём в переменную api ключ

In [ ]:
from nixtlats import TimeGPT # импортируем некоторые необходимые библиотеки

timegpt = TimeGPT(token = api_key) # проводим идентификацию ключа

/usr/local/lib/python3.10/dist-packages/nixtlats/__init__.py:5: FutureWarning: This package is deprecated, please install nixtla instead.
  warnings.warn("This package is deprecated, please install nixtla instead.", category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/nixtlats/nixtla_client.py:53: FutureWarning: `'token'` is deprecated; use `'api_key'` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nixtlats/nixtla_client.py:1604: FutureWarning: Class `TimeGPT` is deprecated; use `NixtlaClient` instead.
  warnings.warn(


In [ ]:
ETTh1 = pd.read_csv('/content/ETDataset/ETT-small/ETTh1.csv')
ETTh2 = pd.read_csv('/content/ETDataset/ETT-small/ETTh2.csv')
ETTm1 = pd.read_csv('/content/ETDataset/ETT-small/ETTm1.csv')
ETTm2 = pd.read_csv('/content/ETDataset/ETT-small/ETTm2.csv')
exchange_rate = pd.read_csv('/content/exchange_rate/exchange_rate.csv')
illness = pd.read_csv('/content/illness/national_illness.csv')
weather = pd.read_csv('/content/weather/weather.csv')

In [ ]:
def TimeGPT_func_prep(df, ds, y):
  '''обработка наборов данных по правилам Nixtla'''
  df = df.rename(columns={ds: 'ds', y: 'y'}) # по правилам Nixtla
  df['ds'] = pd.to_datetime(df['ds'])
  df.index.name = 'unique_id' #
  df.reset_index(inplace=True) #
  df['unique_id'] = 1
  last_column = df.iloc[:, -1] # копируем последний столбец
  df = df.iloc[:, :-1] # удаляем последний столбец из исходного DataFrame
  df.insert(2, 'y', last_column) # вставляем скопированный столбец на вторую позицию (индекс 1)
  return df


In [ ]:
df_list = [ETTh1, ETTh2, ETTm1, ETTm2,  exchange_rate, illness, weather]
for i, df in enumerate(df_list):
    if i == 0:
        ETTh1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 1:
        ETTh2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 2:
        ETTm1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 3:
        ETTm2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 4:
        exchange_rate = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 5:
        illness = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 6:
        weather = TimeGPT_func_prep(df, 'date', 'OT')

In [ ]:
def train_test_func(df):
    train_size = int(len(df) * 0.8)
    test_size = len(df) - train_size
    X_train, X_test = df[0:train_size], df[train_size:len(df)]
    return X_train, X_test

# Создание словаря для хранения результатов
split_data = {}

# Применение функции к каждому датасету из списка df_list
for df_name, df in zip(['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'exchange_rate', 'illness', 'weather'], df_list):
    X_train, X_test = train_test_func(df)
    split_data[f'{df_name}_train'] = X_train
    split_data[f'{df_name}_test'] = X_test

# Вывод размеров полученных выборок для каждого датасета
for key, value in split_data.items():
    print(f"Размер {key}: {len(value)}")


Размер ETTh1_train: 13936
Размер ETTh1_test: 3484
Размер ETTh2_train: 13936
Размер ETTh2_test: 3484
Размер ETTm1_train: 55744
Размер ETTm1_test: 13936
Размер ETTm2_train: 55744
Размер ETTm2_test: 13936
Размер exchange_rate_train: 6070
Размер exchange_rate_test: 1518
Размер illness_train: 772
Размер illness_test: 194
Размер weather_train: 42156
Размер weather_test: 10540


In [ ]:
ETTh1_train, ETTh1_test = train_test_func(ETTh1)
ETTh2_train, ETTh2_test = train_test_func(ETTh2)
ETTm1_train, ETTm1_test = train_test_func(ETTm1)
ETTm2_train, ETTm2_test = train_test_func(ETTm2)
exchange_rate_train, exchange_rate_test = train_test_func(exchange_rate)
illness_train, illness_test = train_test_func(illness)
weather_train, weather_test = train_test_func(weather)

In [ ]:
ETTh1_train_uni, ETTh1_test_uni = train_test_func(ETTh1[['ds','y']]) # TimeGPT работает только с одномерными временными рядами

In [ ]:
ETTh1_train_uni

,ds,y
0,2016-07-01 00:00:00,30.531000
1,2016-07-01 01:00:00,27.787001
2,2016-07-01 02:00:00,27.787001
3,2016-07-01 03:00:00,25.044001
4,2016-07-01 04:00:00,21.948000
...,...,...
13931,2018-02-01 11:00:00,4.713000
13932,2018-02-01 12:00:00,4.291000
13933,2018-02-01 13:00:00,4.291000
13934,2018-02-01 14:00:00,6.261000


In [ ]:
df = timegpt.forecast(df=ETTh1_train_uni,h=720, model='timegpt-1-long-horizon', freq='h') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh1.csv")

In [ ]:
step_forecasting = [24, 96, 192, 336, 720]

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh1_test_uni['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('zero-shot ETTh1'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTh1         
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.576 | 0.522 |
|       96       | 0.894 | 1.357 |
|      192       |  0.89 | 1.219 |
|      336       | 1.124 | 2.262 |
|      720       | 1.328 | 2.941 |
+----------------+-------+-------+


## предсказание без дообучения

#### ETTh1

 любыми способами, не получилось добиться от TimeGPT прогнозирвоание многомерного временного ряда или прогнозирование с использованием ковариант
 он всё время выполнял одномерный прогноз

In [ ]:
df = timegpt.forecast(df=ETTh1_train,h=720, model='timegpt-1-long-horizon', freq='h') # любыми способоами, у меня не получилось добиться от
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh1.csv")

In [ ]:
step_forecasting = [24, 96, 192, 336, 720]

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('zero-shot ETTh1'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTh1         
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.576 | 0.522 |
|       96       | 0.894 | 1.357 |
|      192       |  0.89 | 1.219 |
|      336       | 1.124 | 2.262 |
|      720       | 1.328 | 2.941 |
+----------------+-------+-------+


#### ETTh2

In [ ]:
df = timegpt.forecast(df=ETTh2_train,h=720, model='timegpt-1-long-horizon',  freq='h') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh2.csv")

In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('zero-shot ETTh2'.center(len(table.field_names) * 11))
print(table)

13936    14.272500
13937    12.954000
13938    11.196500
13939     9.218500
13940     7.900500
           ...    
17415    47.084999
17416    48.183498
17417    48.183498
17418    46.865501
17419    45.986500
Name: y, Length: 3484, dtype: float64
+----------------+-------+-------+-------+
| Шагов прогноза | sMAPE |  MAE  |  MSE  |
+----------------+-------+-------+-------+
|       24       | 0.159 | 0.576 | 0.522 |
|       96       | 0.361 | 0.894 | 1.357 |
|      192       | 0.316 |  0.89 | 1.219 |
|      336       | 0.329 | 1.124 | 2.262 |
|      720       | 0.378 | 1.328 | 2.941 |
+----------------+-------+-------+-------+


/usr/local/lib/python3.10/dist-packages/sktime/performance_metrics/forecasting/_classes.py:466: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/usr/local/lib/python3.10/dist-packages/sktime/performance_metrics/forecasting/_classes.py:478: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/usr/local/lib/python3.10/dist-packages/sktime/performance_metrics/forecasting/_classes.py:466: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/usr/local/lib/python3.10/dist-packages/sktime/performance_metrics/forecasting/_classes.py:478: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect obj

#### ETTm1

In [ ]:
df = timegpt.forecast(df=ETTm1_train,h=720, model='timegpt-1-long-horizon',freq='15min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm1.csv")

In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot ETTm1'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTm1         
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.435 | 0.254 |
|       96       | 0.392 | 0.236 |
|      192       | 1.399 | 3.602 |
|      336       | 1.337 | 3.054 |
|      720       | 1.173 | 2.287 |
+----------------+-------+-------+


#### ETTm2

In [ ]:
%%time
df = timegpt.forecast(df=ETTm2_train,h=720, model='timegpt-1-long-horizon',freq='15min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm2.csv")

CPU times: user 1.92 s, sys: 14.2 ms, total: 1.94 s
Wall time: 8.36 s


In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot ETTm2'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTm2         
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       |  0.37 | 0.224  |
|       96       | 1.681 | 6.262  |
|      192       | 2.077 |  7.53  |
|      336       | 2.423 | 10.006 |
|      720       | 3.394 | 19.585 |
+----------------+-------+--------+


#### exchange_rate

In [ ]:
%%time
df = timegpt.forecast(df=exchange_rate_train, h=720, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("exchange_rate.csv")

CPU times: user 226 ms, sys: 10.9 ms, total: 236 ms
Wall time: 7.66 s


In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = exchange_rate_train['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot exchange_rate'.center(len(table.field_names) * 11))
print(table)

     zero-shot exchange_rate     
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.224 |  0.05 |
|       96       | 0.247 | 0.061 |
|      192       | 0.253 | 0.064 |
|      336       | 0.263 | 0.069 |
|      720       | 0.303 | 0.094 |
+----------------+-------+-------+


#### weather

In [ ]:
weather1 = weather.drop_duplicates() # иначе выдают ошибку
weather_train, weather_test = train_test_func(weather1)

In [ ]:
weather.columns[3::]

Index(['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
       'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m�)',
       'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)'],
      dtype='object')

In [ ]:
weather[weather.columns[3::]]

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC)
0,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,5.49,1280.62,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45
1,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,5.45,1280.33,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51
2,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,5.43,1280.29,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60
3,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,5.37,1281.97,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70
4,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,5.42,1282.08,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52691,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,5.89,1234.61,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40
52692,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,6.05,1235.20,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42
52693,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,5.93,1235.82,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45
52694,978.26,2.07,276.95,-0.77,81.4,7.09,5.77,1.32,3.68,5.90,1235.49,0.57,1.07,196.6,0.0,0.0,0.0,0.0,0.0,13.47


In [ ]:
weather.drop('y',axis=1)

,unique_id,ds,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),...,rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC)
0,1,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,...,1280.62,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45
1,1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,...,1280.33,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51
2,1,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,...,1280.29,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60
3,1,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,...,1281.97,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70
4,1,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,...,1282.08,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52691,1,2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,...,1234.61,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40
52692,1,2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,...,1235.20,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42
52693,1,2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,...,1235.82,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45
52694,1,2020-12-31 23:50:00,978.26,2.07,276.95,-0.77,81.4,7.09,5.77,1.32,...,1235.49,0.57,1.07,196.6,0.0,0.0,0.0,0.0,0.0,13.47


In [ ]:
%%time
df = timegpt.forecast(df=weather_train, h=720, model='timegpt-1-long-horizon',freq='10min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("weather2.csv")

CPU times: user 2.21 s, sys: 41.5 ms, total: 2.25 s
Wall time: 9.67 s


In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = weather_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot weather'.center(len(table.field_names) * 11))
print(table)

        zero-shot weather        
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 7.942 | 93.442 |
|       96       |  7.51 | 92.097 |
|      192       | 6.104 | 60.032 |
|      336       | 6.233 | 61.101 |
|      720       | 6.411 | 63.333 |
+----------------+-------+--------+


#### illnes

In [ ]:
step_forecasting1 = [24,36,48,60]

In [ ]:
illness

,unique_id,ds,y,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 5-24,ILITOTAL,NUM. OF PROVIDERS
0,1,2002-01-01,176569,1.222620,1.166680,582,805,2060,754
1,1,2002-01-08,186355,1.333440,1.216500,683,872,2267,785
2,1,2002-01-15,192469,1.319290,1.130570,642,878,2176,831
3,1,2002-01-22,207512,1.494840,1.252460,728,1045,2599,863
4,1,2002-01-29,223208,1.471950,1.302370,823,1189,2907,909
...,...,...,...,...,...,...,...,...,...
961,1,2020-06-02,1525058,0.839059,0.846722,2756,3528,12913,3258
962,1,2020-06-09,1538038,0.895958,0.908885,3203,3778,13979,3254
963,1,2020-06-16,1528103,0.910926,0.941625,3478,3796,14389,3177
964,1,2020-06-23,1542813,0.946945,0.972185,3734,3818,14999,3066


In [ ]:
%%time
df = timegpt.forecast(df=illness_train, h=60,freq='7d' ) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ill.csv")

CPU times: user 85.5 ms, sys: 7.4 ms, total: 92.9 ms
Wall time: 4.29 s


In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = illness_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting1:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot illness'.center(len(table.field_names) * 11))
print(table)

        zero-shot illness        
+----------------+------------+------------------+
| Шагов прогноза |    MAE     |       MSE        |
+----------------+------------+------------------+
|       24       | 224384.208 | 79674712630.944  |
|       36       | 286680.894 | 111297762147.064 |
|       48       | 282778.878 | 102641796745.375 |
|       60       | 265048.321 | 89781749298.283  |
+----------------+------------+------------------+


наблюдаю абсолютную невозможность предсказать illness

## с дообучением на 200 точках
значительно больше 200 точек не получалось взять

#### ETTh1 few200

In [ ]:
%%time
df = timegpt.forecast(df=ETTh1_train,h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания

CPU times: user 567 ms, sys: 24.5 ms, total: 591 ms
Wall time: 27.1 s


In [ ]:
predict.to_csv("ETTh1_few200.csv")

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTh1'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTh1       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 1.162 | 1.798 |
|       96       | 0.733 | 0.871 |
|      192       | 0.763 | 0.889 |
|      336       | 1.498 | 4.188 |
|      720       | 2.173 | 7.116 |
+----------------+-------+-------+


#### ETTh2 few 200

In [ ]:
%%time
df = timegpt.forecast(df=ETTh2_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh2_few200.csv")

CPU times: user 805 ms, sys: 30.7 ms, total: 836 ms
Wall time: 31.3 s


In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTh2'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTh2       
+----------------+-------+---------+
| Шагов прогноза |  MAE  |   MSE   |
+----------------+-------+---------+
|       24       | 2.459 |  9.455  |
|       96       | 2.676 |  11.253 |
|      192       |  3.32 |  16.305 |
|      336       | 5.452 |  45.52  |
|      720       | 9.295 | 116.418 |
+----------------+-------+---------+


### ETTm1 few 200

In [ ]:
df = timegpt.forecast(df=ETTm1_train,h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm1_few200.csv")

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTm1'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm1       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.504 | 0.323 |
|       96       | 0.748 | 0.697 |
|      192       | 0.868 |  1.02 |
|      336       | 0.963 | 1.388 |
|      720       | 0.965 | 1.324 |
+----------------+-------+-------+


### ETTm2 few 200

In [ ]:
df = timegpt.forecast(df=ETTm2_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm2_few200.csv")

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTm2'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm2       
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 2.507 | 7.497  |
|       96       | 4.129 | 20.084 |
|      192       | 3.817 | 17.049 |
|      336       | 2.703 |  10.6  |
|      720       | 2.372 | 9.018  |
+----------------+-------+--------+


### exchange_rate few 200

In [ ]:
df = timegpt.forecast(df=exchange_rate_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("exchange_rate_few200.csv")

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = exchange_rate_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 exchange_rate'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm2       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.008 |  0.0  |
|       96       | 0.013 |  0.0  |
|      192       | 0.035 | 0.002 |
|      336       | 0.045 | 0.003 |
|      720       | 0.098 | 0.013 |
+----------------+-------+-------+


### weather few 200

In [ ]:
weather1 = weather.drop_duplicates() # иначе выдают ошибку
weather_train, weather_test = train_test_func(weather1)

In [ ]:
%%time
df = timegpt.forecast(df=weather_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon',freq='10min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("weather200.csv")

CPU times: user 1.51 s, sys: 37 ms, total: 1.55 s
Wall time: 55 s


In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = weather_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot weather'.center(len(table.field_names) * 11))
print(table)

        zero-shot weather        
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 5.963 | 47.185 |
|       96       |  8.6  | 98.665 |
|      192       | 6.153 | 59.597 |
|      336       | 5.265 | 43.522 |
|      720       | 5.494 | 47.21  |
+----------------+-------+--------+


### illness few 200

In [ ]:
df = timegpt.forecast(df=illness_train, h=60, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("illness_few200.csv")

In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = illness_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting1:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+------------+-----------------+
| Шагов прогноза |    MAE     |       MSE       |
+----------------+------------+-----------------+
|       24       | 82157.451  | 11309856808.086 |
|       36       | 101026.809 | 14891674134.025 |
|       48       | 103290.525 |  14561701775.17 |
|       60       | 119798.829 | 18689907972.032 |
+----------------+------------+-----------------+
